In [21]:
import numpy as np
import glob
import os
import polars as pl

from natsort import natsorted
from tqdm import tqdm

def create_xyz_file(data_path,output_file):
    # data = np.loadtxt(data_path, dtype=float, usecols=[0, 1, 2, 3, 4, 5,6,7,8])
    data = np.loadtxt(data_path, dtype=float, usecols=[0, 1, 2, 3, 4, 5])

    # Extract the required columns
    x, y, z = data[:, 0], data[:, 1], data[:, 2]
    px, py, pz = data[:, 3], data[:, 4], data[:, 5]

    # Calculate max values
    max_x = np.max(x)+1
    max_y = np.max(y)+1
    max_z = np.max(z)+1

    # Mapping
    color_map = {
        0: [0],
        1: [1],
        2: [2],
        3: [3],
        4: [4],
        5: [5]
    }

    # Basis vectors
    base_vectors = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [-1, 0, 0], [0, -1, 0], [0, 0, -1]])

    # Normarize the polarization vectors
    p_vectors = np.vstack((px, py, pz)).T
    p_norm = np.linalg.norm(p_vectors, axis=1, keepdims=True)
    unit_vectors = p_vectors / p_norm

    # Calculate the dot product of the unit vectors and the base vectors
    base_norm = np.linalg.norm(base_vectors, axis=1, keepdims=True)
    unit_base_vectors = base_vectors / base_norm
    dot_products = np.dot(unit_vectors, unit_base_vectors.T)
    max_indices = np.argmax(dot_products, axis=1)

    # output_file = data_path.replace('.coord', 're.xyz')
    output_file = output_file+"/"+ os.path.basename(data_path).replace('.coord', 're2.xyz')
    # Make a new XYZ file
    with open(output_file, 'w') as f:
        f.write(f"{len(x)}\n")
        f.write(f'Lattice="{max_x} 0.0 0.0 0.0 {max_y} 0.0 0.0 0.0 {max_z}" Properties=pos:R:3:velo:R:3:structure_type:S:1\n')
        for i in range(len(x)):
            color = color_map[max_indices[i]]
            f.write(f"{int(x[i])} {int(y[i])} {int(z[i])} {px[i]:.3f} {py[i]:.3f} {pz[i]:.3f} {color[0]}\n")
            

def create_xyz_file_polars(data_path,output_file):
    # データの読み込み
    data = np.loadtxt(data_path, dtype=float, usecols=[0, 1, 2, 3, 4, 5])
    df = pl.DataFrame(data)
    df.columns = ['x', 'y', 'z', 'px', 'py', 'pz']

    # 最大値の計算
    max_x = df['x'].max() + 1
    max_y = df['y'].max() + 1
    max_z = df['z'].max() + 1

    # ベースベクトルの定義
    base_vectors = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [-1, 0, 0], [0, -1, 0], [0, 0, -1]])

    # 位置ベクトルの正規化
    df = df.with_columns([
        (pl.col('px') ** 2 + pl.col('py') ** 2 + pl.col('pz') ** 2).sqrt().alias('p_norm')
    ])
    df = df.with_columns([
        (pl.col('px') / pl.col('p_norm')).alias('unit_x'),
        (pl.col('py') / pl.col('p_norm')).alias('unit_y'),
        (pl.col('pz') / pl.col('p_norm')).alias('unit_z')
    ])

    # ドット積の計算とインデックスの取得
    dot_products = np.dot(df.select(['unit_x', 'unit_y', 'unit_z']).to_numpy(), base_vectors.T)
    max_indices = np.argmax(dot_products, axis=1)

    # 構造タイプの割り当て
    df = df.with_columns(pl.Series(max_indices).alias('structure_type'))
    
    # output_file = data_path.replace('.coord', 're.xyz')
    output_file = output_file+"/"+ os.path.basename(data_path).replace('.coord', 're2.xyz')
    # ヘッダー情報の書き込み
    with open(output_file, 'w') as f:
        f.write(f"{len(df)}\n")
        f.write(f'Lattice="{max_x} 0.0 0.0 0.0 {max_y} 0.0 0.0 0.0 {max_z}" Properties=pos:R:3:velo:R:3:structure_type:S:1\n')
    
    # データの書き込み
    # 一時的にデータを文字列に変換
    data_str = df.select([
        pl.col('x').cast(pl.Int32),
        pl.col('y').cast(pl.Int32),
        pl.col('z').cast(pl.Int32),
        pl.col('px').round(3),
        pl.col('py').round(3),
        pl.col('pz').round(3),
        pl.col('structure_type')
    ]).write_csv(separator=' ', include_header=False)

    # 文字列をファイルに追記
    with open(output_file, 'a') as f:
        f.write(data_str)

data_path = r"C:\Users\kishihama\Laboratory\Papers\Works\1_feram_BulkvsFilm_Hys\data\cooling001-900K.0000050000.coord"
output_file = r"C:\Users\kishihama\Laboratory\Papers\Works\1_feram_BulkvsFilm_Hys\data"
create_xyz_file_polars(data_path,output_file)

In [ ]:
data_path_origin = r"E:\Simulation_Backup\feram\17PbTiO3Phase\001_thickdep-old\001_vs\016_bulk128-128-128\1_900to300K5e4_-05GPa_vs-old\1_E030\coord_files\100-"
output_file = r"E:\Simulation_Backup\feram\17PbTiO3Phase\001_thickdep-old\001_vs\016_bulk128-128-128\1_900to300K5e4_-05GPa_vs-old\1_E030\coord_files\016_bulk128-128-128_1_900to300K5e4_-05GPa_vs-old_300K"
files_list = natsorted(glob.glob(os.path.join(data_path_origin, '*.coord')))
# files_list = [f for f in files_list if ".000" in f]
files_list = [f for f in files_list if "sin-PT300K.000" in f]

for i in tqdm(range(len(files_list))):
    # create_xyz_file(files_list[i],output_file)
    create_xyz_file_polars(files_list[i],output_file)

In [ ]:
data_path_origin = r"C:\Users\kishihama\Laboratory\ISSP\17PbTiO3-phase\017_Thickness\011_128-128-128_7_900to300K_4e4_15per-old\011_128-128-128_7_900to300K_4e4_15per-old_x"
files_list = natsorted(glob.glob(os.path.join(data_path_origin, '*.coord')))
# files_list = [f for f in files_list if ".000" in f]
files_list = [f for f in files_list if "sin-PT300K.000" in f]
for i in tqdm(range(len(files_list))):
    create_xyz_file(files_list[i])

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgb
from PIL import Image

map_name= 'full_new_241125'
map_path = r'C:\Users\kishihama\Laboratory\Programming\Python\feram\data\custom_color_map'+map_name+'.png'
# カラーマップの定義
color_map = {
    0: "violet",  # [100]
    1: "cyan",  # [010]
    2: "limegreen",  # [001]
    3: "red",  # [-100] #
    4: "blue",  # [0-10]
    5: "orange"  # [00-1]
}
color_map = {
    0: "red",  # [100]
    1: "blue",  # [010]
    2: "limegreen",  # [001]
    3: "red",  # [-100] #
    4: "blue",  # [0-10]
    5: "orange"  # [00-1]
}
color_map = {
    0: "limegreen",  # [100]
    1: "orange",  # [010]
    2: "mediumblue",  # [001]
    3: "limegreen",  # [-100] #
    4: "orange",  # [0-10]
    5: "crimson"  # [00-1]
}
color_map = {
    0: "lime",  # [100]
    1: "orange",  # [010]
    2: "mediumblue",  # [001]
    3: "green",  # [-100] #
    4: "gold",  # [0-10]
    5: "crimson"  # [00-1]
}
# 色の数
num_colors = len(color_map)

# 幅1ピクセル、高さ100ピクセルの画像を作成
color_image = np.zeros((1, num_colors, 3), dtype=np.uint8)

# カラー画像にデータを入力
for i, (key, color) in enumerate(color_map.items()):
    color_rgb = to_rgb(color)  # 色名をRGBに変換
    color_image[0, i] = [int(c * 255) for c in color_rgb]  # 0-255のuint8にスケーリング

# カラーマップを表示するための図を作成
plt.figure(figsize=(num_colors, 1))
plt.imshow(color_image, aspect='auto')
plt.axis('off')  # 軸を非表示
plt.savefig(map_path, dpi=300, bbox_inches='tight')
plt.close()

image = Image.open(map_path)

# 画像をNumPy配列に変換
image_array = np.array(image)

# 白い部分を特定 (閾値を設定)
threshold = 200  # RGB値が200以上の場合、白とみなす
mask = (image_array[..., 0] > threshold) & (image_array[..., 1] > threshold) & (image_array[..., 2] > threshold)

# 白くない部分の座標を取得
coords = np.column_stack(np.where(~mask))

# トリミングボックスを取得
if coords.size > 0:
    x0, y0 = coords.min(axis=0)
    x1, y1 = coords.max(axis=0)

    # 画像をトリミング
    trimmed_image = image.crop((y0, x0, y1 + 1, x1 + 1))  # 座標に基づいてトリミング
else:
    trimmed_image = image  # トリミングする部分が見つからない場合は元の画像を使用
# output_file = map_path.replace('.png', 'trimmed_image.png')
# トリミングした画像を保存
trimmed_image.save(map_path)
trimmed_image.show()

print('トリミングした画像が保存されました: '+map_path)

トリミングした画像が保存されました: C:\Users\kishihama\Laboratory\Programming\Python\feram\data\custom_color_mapfull_new_241125.png


In [14]:
from ovito.io import import_file
from ovito.vis import Viewport, ParticlesVis
from ovito.modifiers import SliceModifier, ColorCodingModifier, DeleteSelectedModifier, SelectTypeModifier

from PIL import Image

def render_image(input_path, output_path, map_path, type_name, delete_types, particle_size=1.0, show_image=True):
    pipeline = import_file(input_path)
    # Append modifiers
    pipeline.modifiers.append(SliceModifier(distance=63, normal=(0,0,1)))
    pipeline.modifiers.append(ColorCodingModifier(property=type_name, start_value=0, end_value=5, gradient=ColorCodingModifier.Image(map_path)))
    pipeline.modifiers.append(SelectTypeModifier(operate_on='particles', property=type_name, types=delete_types))
    pipeline.modifiers.append(DeleteSelectedModifier())
    # Set the color of the simulation cell
    pipeline.source.data.cell.vis.enabled = False

    # Change particle shape to cube and set size
    particles_vis = pipeline.source.data.particles.vis
    particles_vis.shape = ParticlesVis.Shape.Sphere #Box
    particles_vis.radius = particle_size

    # Append to the scene
    pipeline.add_to_scene()

    # Set the viewport
    vp = Viewport()
    vp.type = Viewport.Type.Perspective
    vp.camera_dir = (-0.5, 1, -0.5)
    vp.camera_pos = (0, 0, 0)
    vp.zoom_all()

    # Render the image
    vp.render_image(size=(800, 600), filename=output_path, background=(1, 1, 1))

    if show_image:
        # Show the image
        img = Image.open(output_path)
        img.show()

    pipeline.remove_from_scene()

# import the file
input_path = r"C:\Users\kishihama\Laboratory\ISSP\17PbTiO3-phase\017_Thickness\011_128-128-128_7_900to300K_4e4_15per-old\011_128-128-128_7_900to300K_4e4_15per-old_x120re.xyz"
output_path = "./images/ovito_test4.png"
map_name = 'simple_4_241112'
map_path = r'C:\Users\kishihama\Laboratory\Programming\Python\feram\data\custom_color_map' + map_name + '.png'
type_name = 'Structure Type'
delete_types = {"Type 0", "Type 1", "Type 2"}
delete_types = {"Type 2"}
render_image(input_path, output_path, map_path, type_name, delete_types, particle_size=0.7)

In [8]:
data_path = r"C:\Users\kishihama\Laboratory\ISSP\17PbTiO3-phase\017_Thickness\00_gradtest\3_2per_grad-epit_2_64-64-512_a15b15cooling002-300K.0000020000.coord"
# create_xyz_file(data_path)
input_path = data_path.replace('.coord', '_st.xyz')
output_path = data_path.replace('.coord', '.png')
map_name= 'simple_4_241112'
map_path = r'C:\Users\kishihama\Laboratory\Programming\Python\feram\data\custom_color_map'+map_name+'.png'
type_name= 'Structure Type'
delete_types = {"Type 0","Type 2"}
delete_types = {}
render_image(input_path,output_path, map_path, type_name, delete_types)

In [24]:
from ovito.io import import_file
from ovito.vis import Viewport, TachyonRenderer, ParticlesVis
from ovito.modifiers import SliceModifier, ColorCodingModifier, DeleteSelectedModifier, SelectTypeModifier

from PIL import Image
import os,glob,math
from natsort import natsorted
from tqdm import tqdm
from ovito.io import import_file
from ovito.vis import Viewport, ParticlesVis
from ovito.modifiers import SliceModifier, ColorCodingModifier, DeleteSelectedModifier, SelectTypeModifier

from PIL import Image

def render_image(input_path, output_path, map_path, type_name, delete_types,slice_z,size_x, size_y, particle_size=1.0, show_image=True):
    pipeline = import_file(input_path)
    # Append modifiers
    pipeline.modifiers.append(SliceModifier(distance=slice_z, normal=(0,0,1)))
    pipeline.modifiers.append(ColorCodingModifier(property=type_name, start_value=0, end_value=5, gradient=ColorCodingModifier.Image(map_path)))
    pipeline.modifiers.append(SelectTypeModifier(operate_on='particles', property=type_name, types=delete_types))
    pipeline.modifiers.append(DeleteSelectedModifier())
    # Set the color of the simulation cell
    pipeline.source.data.cell.vis.enabled = False

    # Change particle shape to cube and set size
    particles_vis = pipeline.source.data.particles.vis
    particles_vis.shape = ParticlesVis.Shape.Box #Box Sphere
    particles_vis.radius = particle_size

    # Append to the scene
    pipeline.add_to_scene()

    # Set the viewport
    vp = Viewport()
    vp.type = Viewport.Type.Perspective
    vp.camera_dir = (-0.5, 1, -0.5)
    vp.camera_pos = (50,0,0)

    vp.zoom_all(size=(size_x, size_y))

    # Configure Tachyon renderer for high-quality rendering
    renderer = TachyonRenderer()
    renderer.ambient_occlusion = True
    renderer.ambient_occlusion_brightness = 0.8
    renderer.shadows = True
    renderer.direct_light_intensity = 1.2
    renderer.antialiasing = True

    # Render the image with high resolution
    vp.render_image(size=(size_x,size_y), filename=output_path, alpha=True, renderer=renderer) #, background=(1, 1, 1)

    if show_image:
        # Show the image
        img = Image.open(output_path)
        img.show()

    pipeline.remove_from_scene()
    
# import the file
input_path = r"C:\Users\kishihama\Laboratory\Papers\Works\1_feram_BulkvsFilm_Hys\data\016_bulk128-128-128_1_900to300K5e4_-05GPa_vs-old_300Kre2.xyz"
# output_path = "../images/ovito_test4.png"
size_x, size_y = 1300, 1024
output_path = input_path.replace('.xyz', 'sas'+str(size_x)+'-'+str(size_y)+'.png')
map_name= 'simple_4_241112'
map_path = r'C:\Users\kishihama\Laboratory\Programming\Python\feram\data\custom_color_map'+map_name+'.png'
type_name= 'Structure Type'
slice_z = 500
delete_types = {"Type 0","Type 2"}
delete_types = {}
render_image(input_path,output_path, map_path, type_name, delete_types,slice_z,size_x,size_y, particle_size=0.45,show_image=True)


In [29]:
from ovito.io import import_file
from ovito.vis import Viewport, TachyonRenderer, ParticlesVis
from ovito.modifiers import SliceModifier, ColorCodingModifier, DeleteSelectedModifier, SelectTypeModifier

from PIL import Image
import os,glob,math
from natsort import natsorted
from tqdm import tqdm
from ovito.io import import_file
from ovito.vis import Viewport, ParticlesVis
from ovito.modifiers import SliceModifier, ColorCodingModifier, DeleteSelectedModifier, SelectTypeModifier

from PIL import Image

def render_image(input_path, output_path, map_path, type_name, delete_types,slice_z,size_x, size_y, particle_size=1.0, show_image=True):
    pipeline = import_file(input_path)
    # Append modifiers
    pipeline.modifiers.append(SliceModifier(distance=slice_z, normal=(0,0,1)))
    pipeline.modifiers.append(ColorCodingModifier(property=type_name, start_value=0, end_value=5, gradient=ColorCodingModifier.Image(map_path)))
    pipeline.modifiers.append(SelectTypeModifier(operate_on='particles', property=type_name, types=delete_types))
    pipeline.modifiers.append(DeleteSelectedModifier())
    # Set the color of the simulation cell
    # pipeline.source.data.cell.vis.enabled = False

    # Change particle shape to cube and set size
    particles_vis = pipeline.source.data.particles.vis
    particles_vis.shape = ParticlesVis.Shape.Box #Box Sphere
    particles_vis.radius = particle_size

    # Append to the scene
    pipeline.add_to_scene()

    # Set the viewport
    vp = Viewport()
    vp.type = Viewport.Type.Ortho #Perspective
    # vp.camera_dir = (-0.5, 1, -0.5)
    vp.camera_dir = (-0.3, 0.9, -0.3)
    vp.camera_pos = (0,0,0)

    vp.zoom_all(size=(size_x, size_y))

    # Configure Tachyon renderer for high-quality rendering
    renderer = TachyonRenderer()
    renderer.ambient_occlusion = True
    renderer.ambient_occlusion_brightness = 0.8
    renderer.shadows = True
    renderer.direct_light_intensity = 1.2
    renderer.antialiasing = True

    # Render the image with high resolution
    vp.render_image(size=(size_x,size_y), filename=output_path, alpha=True, renderer=renderer) #, background=(1, 1, 1)

    if show_image:
        # Show the image
        img = Image.open(output_path)
        img.show()

    pipeline.remove_from_scene()
    
# import the file
input_path = r"C:\Users\kishihama\Laboratory\Papers\Works\1_feram_BulkvsFilm_Hys\data\016_bulk128-128-128_1_900to300K5e4_-05GPa_vs-old_300Kre2.xyz"
# output_path = "../images/ovito_test4.png"
size_x, size_y = 800, 600 #1300, 1024
output_path = input_path.replace('.xyz', 'ortho_cell'+str(size_x)+'-'+str(size_y)+'.png')
map_name= 'simple_4_241112'
map_path = r'C:\Users\kishihama\Laboratory\Programming\Python\feram\data\custom_color_map'+map_name+'.png'
type_name= 'Structure Type'
slice_z = 500
delete_types = {"Type 0","Type 2"}
delete_types = {}
render_image(input_path,output_path, map_path, type_name, delete_types,slice_z,size_x,size_y, particle_size=0.45,show_image=True)
